# 여드름 종류 진단 AI

### 라이브러리 Import

In [4]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

### 데이터 로드 및 전처리

In [5]:
data_dir = "./data/image"  # Kaggle에서 받은 데이터셋 경로
image_size = (224, 224)
labels = ["pustules", "papules", "comedones", "normal"]

def load_images_and_labels():
    images, targets = [], []
    for idx, label in enumerate(labels):
        folder_path = os.path.join(data_dir, label)
        for file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, image_size)
            images.append(img)
            targets.append(idx)
    return np.array(images), np.array(targets)

X, y = load_images_and_labels()
X = X / 255.0  # Normalization
y = tf.keras.utils.to_categorical(y, num_classes=4)

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

FileNotFoundError: [Errno 2] No such file or directory: './data/image/pustules'

### 데이터 증강 (Data Augmentation)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.2
)
datagen.fit(X_train)

### 모델 구성

In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(4, activation='softmax')(x)  # 4개 클래스
model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


### 모델 학습

In [ ]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_val, y_val),
                    epochs=20,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)])

### 모델 평가, 모델 저장 및 배포 설정

In [ ]:
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc:.2f}")

model.save("./acne_classifier.h5")